In [ ]:
import pyodbc
import csv
import time

try:
    # SQL Server Connection Setup with increased timeout
    conn = pyodbc.connect(
        'DRIVER={SQL Server};'
        'SERVER=lds.di.unipi.it;'       
        'DATABASE=Group_ID_7_DB;'       
        'UID=Group_ID_7;'               
        'PWD=SZGJIVBY;'                 
        'Connection Timeout=60;'        
    )
    # 
    # Set the command timeout at the connection level (if supported)
    conn.timeout = 300  # Command timeout set to 5 minutes

    cursor = conn.cursor()

    print("Connected to the SQL Server successfully!")

except pyodbc.Error as e:
    print("Error connecting to SQL Server:", e)

In [ ]:
# Function to insert data from CSV into a table
def insert_data_from_csv(filename, table_name, has_identity=False, batch_size=1000, skip_column=None, skip_value=None):
    with open(filename, 'r') as file:
        reader = csv.reader(file)
        headers = next(reader)  # Get column headers
        placeholders = ', '.join(['?'] * len(headers))
        query = f'INSERT INTO {table_name} ({", ".join(headers)}) VALUES ({placeholders})'

        # Enable IDENTITY_INSERT if required
        if has_identity:
            cursor.execute(f"SET IDENTITY_INSERT {table_name} ON")

        rows_to_insert = []
        for row in reader:
            # Skip rows based on the given condition
            if skip_column and skip_value:
                column_index = headers.index(skip_column)
                if int(row[column_index]) <= skip_value:
                    continue

            rows_to_insert.append(row)
            if len(rows_to_insert) == batch_size:
                try:
                    cursor.executemany(query, rows_to_insert)
                except pyodbc.Error as e:
                    print(f"Error: {e}")
                rows_to_insert = []  # Clear the list after batch insert

        # Insert remaining rows (less than batch size)
        if rows_to_insert:
            try:
                cursor.executemany(query, rows_to_insert)
            except pyodbc.Error as e:
                print(f"Error: {e}")

        # Disable IDENTITY_INSERT if it was enabled
        if has_identity:
            cursor.execute(f"SET IDENTITY_INSERT {table_name} OFF")

        conn.commit()
        print(f"Data inserted into {table_name} successfully from {filename}")

# List of tables and corresponding CSV files
tables = [
    ('DimCause', 'DimCause.csv', True),          # Table with identity column       DONE
    ('DimDate', 'DimDate.csv', True),            # Table with identity column       DONE
    ('DimGeography', 'DimGeography.csv', True),  # Table with identity column       DONE
    ('DimWeather', 'DimWeather.csv', True),      # Table with identity column       DONE       
]

# Main execution
if __name__ == "__main__":
    
    conn = connect_to_database()
    cursor = conn.cursor()

    # Loop through each table and insert data
    for table in tables:
        table_name = table[0]
        filename = table[1]
        has_identity = table[2]

        # Handle optional skip conditions
        skip_column = table[3] if len(table) > 3 else None
        skip_value = table[4] if len(table) > 4 else None

        print(f"Inserting data into {table_name} from {filename}")
        try:
            insert_data_from_csv(
                filename, table_name, has_identity=has_identity, batch_size=500, skip_column=skip_column, skip_value=skip_value
            )
        except Exception as e:
            print(f"Error inserting data into {table_name}: {e}")

    conn.close()
    print("Data insertion for all tables completed successfully!")


In [ ]:
# Function to check if a Person_ID already exists in the table
def check_existing_person_ids(cursor, table_name):
    cursor.execute(f"SELECT Person_ID FROM {table_name}")
    return {row[0] for row in cursor.fetchall()}

# Function to insert data from CSV into DimPerson table
def insert_data_from_csv(filename, table_name, conn, batch_size=1000):
    try:
        cursor = conn.cursor()
        existing_person_ids = check_existing_person_ids(cursor, table_name)
        print(f"Total existing records in {table_name}: {len(existing_person_ids)}")

        with open(filename, 'r') as file:
            reader = csv.reader(file)
            headers = next(reader)  # Get column headers
            placeholders = ', '.join(['?'] * len(headers))
            query = f'INSERT INTO {table_name} ({", ".join(headers)}) VALUES ({placeholders})'

            rows_to_insert = []
            for row in reader:
                # Skip rows with duplicate Person_ID
                person_id_index = headers.index('Person_ID')
                if row[person_id_index] in existing_person_ids:
                    continue

                # Convert 'Age' to integer if it exists in the headers
                if 'Age' in headers:
                    age_index = headers.index('Age')
                    try:
                        row[age_index] = int(float(row[age_index]))  # Convert to integer
                    except ValueError:
                        print(f"Skipping invalid Age value: {row[age_index]}")
                        continue

                rows_to_insert.append(row)
                if len(rows_to_insert) == batch_size:
                    try:
                        cursor.executemany(query, rows_to_insert)
                        conn.commit()
                    except pyodbc.Error as e:
                        print(f"Error during batch insert for {table_name}: {e}")
                    rows_to_insert = []  # Clear the list after batch insert

            # Insert remaining rows
            if rows_to_insert:
                try:
                    cursor.executemany(query, rows_to_insert)
                    conn.commit()
                except pyodbc.Error as e:
                    print(f"Error during final batch insert for {table_name}: {e}")

            print(f"Data inserted into {table_name} successfully from {filename}")

    except Exception as e:
        print(f"Error inserting data into {table_name}: {e}")

# Main execution
if __name__ == "__main__":
    conn = connect_to_database()

    # Specify the table and CSV file for DimPerson
    table_name = 'DimPerson'
    filename = 'DimPerson.csv'

    print(f"Inserting data into {table_name} from {filename}")
    try:
        insert_data_from_csv(filename, table_name, conn, batch_size=500)
    except Exception as e:
        print(f"Error inserting data into {table_name}: {e}")

    conn.close()
    print("Data insertion completed successfully!")


In [ ]:
# Function to fetch existing Vehicle_IDs
def fetch_existing_vehicle_ids(table_name):
    try:
        query = f"SELECT Vehicle_ID FROM {table_name}"
        cursor.execute(query)
        rows = cursor.fetchall()
        return {str(row[0]) for row in rows}  # Return a set of existing Vehicle_IDs
    except pyodbc.Error as e:
        print(f"Error fetching existing Vehicle_IDs: {e}")
        return set()

# Function to insert data from CSV into DimVehicle table
def insert_data_with_identity(filename, table_name, batch_size=1000):
    try:
        # Fetch existing Vehicle_IDs
        existing_vehicle_ids = fetch_existing_vehicle_ids(table_name)

        with open(filename, 'r') as file:
            reader = csv.reader(file)
            headers = next(reader)  # Get column headers
            placeholders = ', '.join(['?'] * len(headers))
            query = f'INSERT INTO {table_name} ({", ".join(headers)}) VALUES ({placeholders})'

            cursor.execute(f"SET IDENTITY_INSERT {table_name} ON")  # Enable IDENTITY_INSERT

            rows_to_insert = []
            for row in reader:
                # Skip rows with duplicate Vehicle_ID
                vehicle_id_index = headers.index('Vehicle_ID')
                if row[vehicle_id_index] in existing_vehicle_ids:
                    continue

                # Convert 'Year' to integer if it exists in the headers
                if 'Year' in headers:
                    year_index = headers.index('Year')
                    try:
                        row[year_index] = int(float(row[year_index]))  # Convert float to integer
                    except ValueError:
                        print(f"Skipping invalid Year value: {row[year_index]}")
                        continue

                rows_to_insert.append(row)
                if len(rows_to_insert) == batch_size:
                    try:
                        cursor.executemany(query, rows_to_insert)
                        conn.commit()
                    except pyodbc.Error as e:
                        print(f"Error during batch insert for {table_name}: {e}")
                    rows_to_insert = []  # Clear the list after batch insert

            # Insert remaining rows
            if rows_to_insert:
                try:
                    cursor.executemany(query, rows_to_insert)
                    conn.commit()
                except pyodbc.Error as e:
                    print(f"Error during final batch insert for {table_name}: {e}")

            cursor.execute(f"SET IDENTITY_INSERT {table_name} OFF")  # Disable IDENTITY_INSERT
            print(f"Data inserted into {table_name} successfully from {filename}")

    except Exception as e:
        print(f"Error inserting data into {table_name}: {e}")

# Main execution
if __name__ == "__main__":
    conn = connect_to_database()
    cursor = conn.cursor()

    # Specify the table and CSV file for DimVehicle
    table_name = 'DimVehicle'
    filename = 'DimVehicle.csv'

    print(f"Inserting data into {table_name} from {filename}")
    try:
        insert_data_with_identity(filename, table_name, batch_size=500)
    except Exception as e:
        print(f"Error inserting data into {table_name}: {e}")

    conn.close()
    print("Data insertion completed successfully!")


In [ ]:
# Function to get existing primary keys from the FactCrash table
def get_existing_fact_crash_ids(table_name):
    try:
        query = f"SELECT Crash_Detail_ID FROM {table_name}"
        cursor.execute(query)
        return {str(row[0]) for row in cursor.fetchall()}  # Return a set of existing Crash_Detail_IDs as strings
    except Exception as e:
        print(f"Error fetching existing records from {table_name}: {e}")
        return set()

# Function to insert data from CSV into FactCrash table with IDENTITY_INSERT
def insert_data_with_identity(filename, table_name, batch_size=1000):
    try:
        # Get the existing Crash_Detail_IDs to skip duplicates
        existing_ids = get_existing_fact_crash_ids(table_name)
        print(f"Fetched {len(existing_ids)} existing IDs from {table_name}.")

        with open(filename, 'r') as file:
            reader = csv.reader(file)
            headers = next(reader)  # Get column headers
            placeholders = ', '.join(['?'] * len(headers))
            query = f'INSERT INTO {table_name} ({", ".join(headers)}) VALUES ({placeholders})'

            # Enable IDENTITY_INSERT for the table
            cursor.execute(f"SET IDENTITY_INSERT {table_name} ON")

            rows_to_insert = []
            for row in reader:
                # Skip rows with duplicate Crash_Detail_ID
                crash_detail_id_index = headers.index('Crash_Detail_ID')
                if str(row[crash_detail_id_index]) in existing_ids:
                    continue  # Skip if ID already exists

                # Convert numeric fields to integers if required
                for col_name in ['num_units', 'Participant_Count', 'Weather_ID', 'Vehicle_ID', 'Date_ID', 'Geography_ID', 'Cause_ID', 'Crash_ID']:
                    if col_name in headers:
                        col_index = headers.index(col_name)
                        try:
                            row[col_index] = int(float(row[col_index]))  # Convert to integer
                        except ValueError:
                            print(f"Skipping invalid value for {col_name}: {row[col_index]}")
                            continue

                rows_to_insert.append(row)
                if len(rows_to_insert) == batch_size:
                    try:
                        cursor.executemany(query, rows_to_insert)
                        conn.commit()
                    except pyodbc.Error as e:
                        print(f"Error during batch insert for {table_name}: {e}")
                    rows_to_insert = []  # Clear the list after batch insert

            # Insert remaining rows
            if rows_to_insert:
                try:
                    cursor.executemany(query, rows_to_insert)
                    conn.commit()
                except pyodbc.Error as e:
                    print(f"Error during final batch insert for {table_name}: {e}")

            # Disable IDENTITY_INSERT after the operation
            cursor.execute(f"SET IDENTITY_INSERT {table_name} OFF")
            print(f"Data inserted into {table_name} successfully from {filename}")

    except Exception as e:
        print(f"Error inserting data into {table_name}: {e}")

# Main execution
if __name__ == "__main__":
    conn = connect_to_database()
    cursor = conn.cursor()

    # Specify the table and CSV file for FactCrash
    table_name = 'FactCrash'
    filename = 'FactCrash.csv'

    print(f"Inserting data into {table_name} from {filename}")
    try:
        insert_data_with_identity(filename, table_name, batch_size=500)
    except Exception as e:
        print(f"Error inserting data into {table_name}: {e}")

    conn.close()
    print("Data insertion completed successfully!")


In [ ]:
# Function to insert data from CSV into a table
def insert_data_from_csv(filename, table_name, has_identity=False, batch_size=1000, retries=3):
    with open(filename, 'r') as file:
        reader = csv.reader(file)
        headers = next(reader)  # Get column headers
        placeholders = ', '.join(['?'] * len(headers))
        query = f'INSERT INTO {table_name} ({", ".join(headers)}) VALUES ({placeholders})'

        # Enable IDENTITY_INSERT if required
        if has_identity:
            cursor.execute(f"SET IDENTITY_INSERT {table_name} ON")

        rows_to_insert = []
        for row in reader:
            # Skip rows with Crash_ID <= 222000
            if int(row[0]) > 222000:  # Assuming row[0] corresponds to Crash_ID
                rows_to_insert.append(row)
            
            # Insert in batches
            if len(rows_to_insert) == batch_size:
                attempt = 0
                while attempt < retries:
                    try:
                        cursor.executemany(query, rows_to_insert)
                        conn.commit()  # Commit the batch
                        break  # Exit retry loop if successful
                    except pyodbc.Error as e:
                        print(f"Error: {e}")
                        attempt += 1
                        print(f"Retrying batch... (Attempt {attempt}/{retries})")
                        time.sleep(2)  # Wait before retrying
                rows_to_insert = []  # Clear the list after batch insert

        # Insert remaining rows (less than batch size)
        if rows_to_insert:
            attempt = 0
            while attempt < retries:
                try:
                    cursor.executemany(query, rows_to_insert)
                    conn.commit()
                    break
                except pyodbc.Error as e:
                    print(f"Error: {e}")
                    attempt += 1
                    print(f"Retrying remaining batch... (Attempt {attempt}/{retries})")
                    time.sleep(2)

        # Disable IDENTITY_INSERT if it was enabled
        if has_identity:
            cursor.execute(f"SET IDENTITY_INSERT {table_name} OFF")

        conn.commit()
        print(f"Data inserted into {table_name} successfully from {filename}")

# List of tables and corresponding CSV files
tables = [
    ('DimCrash', 'DimCrash.csv', True),  # Table with identity column
]

# Main execution
if __name__ == "__main__":
    conn = connect_to_database()
    cursor = conn.cursor()

    # Loop through each table and insert data
    for table_name, filename, has_identity in tables:
        print(f"Inserting data into {table_name} from {filename}")
        try:
            insert_data_from_csv(filename, table_name, has_identity=has_identity, batch_size=500)  # Reduced batch size for better control
        except Exception as e:
            print(f"Error inserting data into {table_name}: {e}")

    conn.close()
    print("Data insertion for all tables completed successfully!")